In [396]:
import pandas as pd
import datetime
from datetime import timedelta
import numpy as np

In [506]:
flights = pd.read_csv('flights_train.csv')

In [507]:
flights.head()

,id,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,ARRIVAL_DELAY
0,0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,2354.0,21.0,15.0,205.0,1448,430,-22.0
1,1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,2.0,12.0,14.0,280.0,2330,750,-9.0
2,2,2015,1,1,4,US,840,N171US,SFO,CLT,20,18.0,16.0,34.0,286.0,2296,806,5.0
3,3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,15.0,15.0,30.0,285.0,2342,805,-9.0
4,4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,24.0,11.0,35.0,235.0,1448,320,-21.0


In [508]:
flights[['SCHEDULED_DEPARTURE', 'SCHEDULED_ARRIVAL', 'SCHEDULED_TIME', 'DEPARTURE_TIME', 'ARRIVAL_DELAY']]

,SCHEDULED_DEPARTURE,SCHEDULED_ARRIVAL,SCHEDULED_TIME,DEPARTURE_TIME,ARRIVAL_DELAY
0,5,430,205.0,2354.0,-22.0
1,10,750,280.0,2.0,-9.0
2,20,806,286.0,18.0,5.0
3,20,805,285.0,15.0,-9.0
4,25,320,235.0,24.0,-21.0
...,...,...,...,...,...
2818548,2359,600,241.0,2358.0,-8.0
2818549,2359,520,201.0,129.0,79.0
2818550,2359,608,249.0,152.0,107.0
2818551,2359,609,250.0,12.0,9.0


In [509]:
def format_hour(chain):
    if pd.isnull(chain):
        return np.nan
    else:
        if chain == 2400: chain = 0
        chain = "{0:04d}".format(int(chain))
        hour = datetime.time(int(chain[0:2]), int(chain[2:4]))
        return hour
    
def combine_date_hour(x):
    if pd.isnull(x[0]) or pd.isnull(x[1]):
        return np.nan
    else:
        return datetime.datetime.combine(x[0],x[1])

def create_flight_time(df, col):    
    list = []
    for index, cols in df[['DATE', col]].iterrows():    
        if pd.isnull(cols[1]):
            list.append(np.nan)
        elif float(cols[1]) == 2400:
            cols[0] += datetime.timedelta(days=1)
            cols[1] = datetime.time(0,0)
            list.append(combine_date_hour(cols))
        else:
            cols[1] = format_hour(cols[1])
            list.append(combine_date_hour(cols))
    return pd.Series(list)

In [510]:
flights['DATE'] = pd.to_datetime(flights[['YEAR','MONTH', 'DAY']])

In [511]:
# flights['SCHEDULED_DEPARTURE'] = create_flight_time(flights, 'SCHEDULED_DEPARTURE')
# flights['DEPARTURE_TIME'] = flights['DEPARTURE_TIME'].apply(format_hour)
# flights['SCHEDULED_ARRIVAL'] = flights['SCHEDULED_ARRIVAL'].apply(format_hour)

In [512]:
# flights.SCHEDULED_DEPARTURE = pd.to_datetime(flights['SCHEDULED_DEPARTURE']).apply(lambda x: x.time())

In [513]:
flights = flights.drop(columns = ['YEAR', 'id', 'FLIGHT_NUMBER', 'TAIL_NUMBER', 'DAY', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'WHEELS_OFF', 'SCHEDULED_ARRIVAL','DATE'])

In [514]:
one_hot = pd.get_dummies(flights.MONTH, prefix = 'Month')
flights = flights.drop('MONTH',axis = 1)
flights = flights.join(one_hot)

In [515]:
one_hot = pd.get_dummies(flights.DAY_OF_WEEK, prefix = 'day_of_week')
flights = flights.drop('DAY_OF_WEEK',axis = 1)
flights = flights.join(one_hot)

In [516]:
one_hot = pd.get_dummies(flights.AIRLINE, prefix = 'AIRLINE')
flights = flights.drop('AIRLINE',axis = 1)
flights = flights.join(one_hot)


In [523]:
flights = flights.drop(columns = ['ORIGIN_AIRPORT', 'DESTINATION_AIRPORT'])

In [535]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [525]:
X = flights
y = flights.ARRIVAL_DELAY

In [526]:
X = X.drop(columns = ['ARRIVAL_DELAY'])

In [527]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [528]:
reg = LinearRegression().fit(X_train, y_train)

In [530]:
reg.score(X_test, y_test)

0.07921363109802171

In [532]:
predicted =reg.predict(X_test)
predicted

array([-1.80752238,  6.12128043,  0.80384238, ...,  2.69441491,
        4.31107443, 24.19296629])

In [536]:
mean_squared_error(predicted, y_test)

1522.7812562944207